# Using Historical mean and std as covariate

In [1]:
from residual_learning.utils import (
                BaseForecaster, 
                ResidualForecasterDarts,
                TimeSeriesPreprocessor,
                crps,
                HistoricalForecaster
)
import pandas as pd
import matplotlib.pyplot as plt
import os
from darts import TimeSeries
from darts.metrics import smape, rho_risk
import numpy as np
import time
from typing import Optional
from darts.utils.likelihood_models import QuantileRegression
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape
from darts.utils.likelihood_models import QuantileRegression
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape
from darts.models import (
                          BlockRNNModel, 
                          TCNModel, 
                          RNNModel, 
                          TransformerModel, 
                          NLinearModel,
                          DLinearModel,
                          NBEATSModel,
                          XGBModel,
                          LinearRegressionModel,
                          TFTModel,
                         )
import optuna
from optuna.integration import PyTorchLightningPruningCallback
import CRPS.CRPS as forecastscore
from datetime import datetime, timedelta


os.environ["CUDA_VISIBLE_DEVICES"] = "1"
targets = pd.read_csv("targets.csv.gz")

data_preprocessor = TimeSeriesPreprocessor()
data_preprocessor.load()

In [2]:
def make_plot(csv_name, site, target_variable, data_preprocessor, record_dict):
    # Loading the forecast csv and creating a time series
    df = pd.read_csv(f"{csv_name}.csv")
    times = pd.to_datetime(df["datetime"])
    times = pd.DatetimeIndex(times)
    values = df.loc[:, df.columns!="datetime"].to_numpy().reshape((len(times), 1, -1))
    model_forecast = TimeSeries.from_times_and_values(times, 
                                                      values, 
                                                      fill_missing_dates=True, freq="D")

    # Presuming data_preprocessor has been called outside of the function
    # (not the best practice), create a validation series from it
    target_series = data_preprocessor.sites_dict[site][target_variable]
    validation_series = target_series.slice(times[0], times[-1]).median()

    # Now, making the forecast based off of historical mean and std
    historical_model = HistoricalForecaster(data_preprocessor=data_preprocessor,
                          output_csv_name="historical_forecaster_output.csv",
                          validation_split_date=str(model_forecast.time_index[0])[:10],
                          forecast_horizon=30,
                          site_id=site,
                          target_variable=target_variable)
    historical_model.make_forecasts()
    
    if site not in record_dict.keys():
        record_dict[site] = {target_variable: {csv_name: model_forecast}}
        record_dict[site][target_variable]["validation"] = validation_series
        # I need to make draw samples from historical would be best to just do this in utils
        record_dict[site][target_variable]["historical"] = historical_model.forecast_ts
    else:
        record_dict[site][target_variable][csv_name] = model_forecast    
    
    validation_series.plot(label="Truth")
    model_forecast.plot(label="Forecast")
    historical_model.forecast_ts.plot(label="historical")

In [19]:
class BaseForecasterHistPlus():
    def __init__(self,
                 model: Optional[str] = None,
                 data_preprocessor: Optional = None,
                 target_variable_column_name: Optional[str] = None,
                 datetime_column_name: Optional[str] = None,
                 covariates_names: Optional[list] = None,
                 output_csv_name: Optional[str] = "residual_forecaster_output.csv",
                 validation_split_date: Optional[str] = None, #YYYY-MM-DD
                 model_hyperparameters: Optional[dict] = None,
                 model_likelihood: Optional[dict] = None,
                 forecast_horizon: Optional[int] = 30,
                 site_id: Optional[str] = None,
                 historical_covariates: Optional[bool] = False,
                 ):
        self.model_ = {"BlockRNN": BlockRNNModel, 
                       "TCN": TCNModel, 
                       "RNN": RNNModel, 
                       "Transformer": TransformerModel,
                       "NLinear": NLinearModel,
                       "DLinear": DLinearModel,
                       "XGB": XGBModel,
                       "NBEATS": NBEATSModel,
                       "Linear": LinearRegressionModel,
                       "TFT": TFTModel}[model]
        self.data_preprocessor = data_preprocessor
        self.target_variable_column_name = target_variable_column_name
        self.datetime_column_name = datetime_column_name
        self.covariates_names = covariates_names
        self.output_csv_name = output_csv_name
        self.validation_split_date = validation_split_date
        self.forecast_horizon = forecast_horizon
        self.site_id = site_id
        if model_hyperparameters == None:
            self.hyperparams = {"input_chunk_length" : 180}
        else:
            self.hyperparams = model_hyperparameters
        self.model_likelihood = model_likelihood
        self.historical_covariates = historical_covariates

        self._preprocess_data()
        
    def _preprocess_data(self):
        """
        Performs gap filling and processing of data into format that
        Darts models will accept
        """
        stitched_series_dict = self.data_preprocessor.sites_dict[self.site_id]

        # If there was failure when doing the GP fit then we can't do preprocessing
        if self.target_variable_column_name in \
                self.data_preprocessor.sites_dict_null[self.site_id]:
            return "Cannot fit this target time series as no GP fit was performed."
        self.inputs = stitched_series_dict[self.target_variable_column_name]

        # And not using the covariates that did not yield GP fits beforehand
        for null_variable in self.data_preprocessor.sites_dict_null[self.site_id]:
            self.covariates_names.remove(null_variable)
            
        # Initializing covariates list then concatenating in for loop
        self.covariates = stitched_series_dict[self.covariates_names[0]]
        for cov_var in self.covariates_names[1:]:
            self.covariates = self.covariates.concatenate(stitched_series_dict[cov_var], 
                                                          axis=1, 
                                                          ignore_time_axis=True)
        
        # Splitting training and validation set
        year = int(self.validation_split_date[:4])
        month = int(self.validation_split_date[5:7])
        day = int(self.validation_split_date[8:])
        split_date = pd.Timestamp(year=year, month=month, day=day)
        self.training_set, self.validation_set = self.inputs.split_before(split_date)
        if self.historical_covariates:
            self.covariates, _ = self.covariates.split_before(split_date)
            self.get_historical_df(self.training_set)
            self.covariates = self.covariates.concatenate(self.historical_samples,
                                                          axis=1,
                                                          ignore_time_axis=True)

    def get_historical_df(self, time_series):
        # Using the medians of the time series
        median_df = time_series.median().pd_dataframe()
        median_df["timestamp"] = pd.to_datetime(median_df.index)
        median_df["day_of_year"] = median_df["timestamp"].dt.dayofyear
        self.median_df = median_df
        
        # Computing average and std for doy's 
        self.doy_df = self.median_df.groupby(['day_of_year'])['0'].agg(['mean', 'std'])
        self.doy_df = self.doy_df.fillna(method="ffill")
        # This produces a new df that gives mean and std for every date (indexed by doy)
        # from the original TimeSeries
        doy_hist_df = self.doy_df.loc[time_series.time_index.dayofyear]

        # Now drawing samples so that we can create a Time Series
        samples = np.array([np.random.normal(doy_hist_df.iloc[i]["mean"], 
                                             doy_hist_df.iloc[i]["std"]/2, 
                                             size=(1,500))
                            for i in range(len(doy_hist_df))])
        # Reconstituting a TimeSeries
        self.historical_samples = TimeSeries.from_times_and_values(
                                                   time_series.time_index,
                                                   samples)

    def tune(self,
             hyperparameter_dict: Optional[dict]
            ):
        """
        Sets up Optuna trial to perform hyperparameter tuning
        Input dictionary will be of the form {"hyperparamter": [values to be tested]}
        """
        # Setting up an optuna Trial
        def objective(trial):
            callback = [PyTorchLightningPruningCallback(trial, monitor="val_loss")]
            hyperparams = {key: trial.suggest_categorical(key, value) 
                                               for key, value in hyperparameter_dict.items()}
        
            model = self.model_(**hyperparams,
                                output_chunk_length=self.forecast_horizon,
                                **self.model_likelihood)
            
            self.scaler = Scaler()
            training_set, covariates = self.scaler.fit_transform([self.training_set,
                                                                  self.covariates])
            # Don't need to tune XGB and linear regression models
            extras = {"past_covariates": covariates,
                      "verbose": False,
                      "epochs": 500}
        
            model.fit(training_set,
                           **extras)
        
            predictions = model.predict(n=len(self.validation_set[:self.forecast_horizon]), 
                                            past_covariates=covariates, 
                                            num_samples=50)
            predictions = self.scaler.inverse_transform(predictions)
            smapes = smape(self.validation_set[:self.forecast_horizon], 
                           predictions, 
                           n_jobs=-1, 
                           verbose=False)
            
            smape_val = np.mean(smapes)
            return smape_val if smape_val != np.nan else float("inf")

        study = optuna.create_study(direction="minimize")
        
        study.optimize(objective, n_trials=50) # Note 10 trials pretty meaningless here
        
        self.hyperparams = study.best_trial.params

    def make_forecasts(self):
        """
        This function fits a Darts model to the training_set
        """
        print(self.hyperparams)
        self.model = self.model_(**self.hyperparams,
                                 output_chunk_length=self.forecast_horizon,
                                 **self.model_likelihood,
                                 random_state=0)
        self.scaler = Scaler()
        training_set, covariates = self.scaler.fit_transform([self.training_set,
                                                              self.covariates])
        # Need to treat XGB and Linear Regression models differently than networks
        extras = {"past_covariates": covariates,
                  "verbose": False,
                  "epochs": 500}
        if self.model_ == XGBModel or self.model_ == LinearRegressionModel:
            del extras["epochs"]
            del extras["verbose"]
        #import pdb; pdb.set_trace()
        extras["past_covariates"] = extras["past_covariates"].median()
        self.model.fit(training_set.median(),
                       **extras)

        predictions = self.model.predict(n=self.forecast_horizon,
                                         past_covariates=covariates.median(), 
                                         num_samples=500)
        predictions = self.scaler.inverse_transform(predictions)

        predictions.pd_dataframe().to_csv(self.output_csv_name)

    def get_historicals_and_residuals(self):
        """
        This function creates a historical forecast along with their residual errors 
        """
        # This presumes that the scaler will not have been modified in interim 
        # from calling `make_forecasts`
        training_set, covariates = self.scaler.transform([self.training_set,
                                                              self.covariates])
        historical_forecasts = self.model.historical_forecasts(
                                            series=training_set,
                                            past_covariates=covariates,
                                            num_samples=500,
                                            forecast_horizon=self.forecast_horizon,
                                            stride=self.forecast_horizon,
                                            retrain=False,
                                            last_points_only=False
                                            )
        historical_forecasts = [self.scaler.inverse_transform(historical_forecast) for
                                                historical_forecast in historical_forecasts]
        # Getting the target time series slice for the historical forecast
        self.historical_ground_truth = self.training_set.slice(
                                            historical_forecasts[0].time_index[0], 
                                            historical_forecasts[-1].time_index[-1])

        # Now concatenating the historical forecasts which were returned
        # as a list above
        self.historical_forecasts = historical_forecasts[0]
        for time_series in historical_forecasts[1:]:
            self.historical_forecasts = self.historical_forecasts.concatenate(time_series, 
                                                                axis=0, 
                                                                ignore_time_axis=True)

        self.residuals = self.historical_ground_truth - self.historical_forecasts

    def make_residuals_csv(self):
        covariates_df = self.covariates.pd_dataframe()
        forecast_df = self.historical_forecasts.pd_dataframe()
        observed_df = self.historical_ground_truth.pd_dataframe()
        residuals_df = self.residuals.pd_dataframe()

        # Creating a folder if it doesn't exist already
        if not os.path.exists(f"{self.model}_residuals/"):
            os.makedirs(f"{self.model}_residuals/")
        # Saving csv's in the **model name**_test directory
        df_dict = {"covariates": block_rnn_forecaster.covariates.pd_dataframe(),
                   "forecast": block_rnn_forecaster.historical_forecasts.pd_dataframe(),
                   "observed": block_rnn_forecaster.historical_ground_truth.pd_dataframe()}
        for variable, df in df_dict.items():
            df.to_csv(f"{self.model}_test/{variable}")

    def plot_by_site(self, site):
        for key in self.sites_dict[site].keys():
            plt.clf()
            self.sites_dict[site][key].plot(color="blue", label=f"{key} @ {site}")
            plt.show()

In [20]:
forecaster_hist_test = BaseForecasterHistPlus(model="BlockRNN",
                                      data_preprocessor=data_preprocessor,
                                      target_variable_column_name="oxygen",
                                      datetime_column_name="datetime",
                                      covariates_names=["air_tmp", "chla", "temperature"],
                                      output_csv_name="trash_block_test.csv",
                                      validation_split_date="2023-03-01",
                                      model_hyperparameters={'input_chunk_length': 540, 
                                                             'hidden_dim': 16, 
                                                             'model': 'GRU', 
                                                             'n_rnn_layers': 4,
                                                             'add_encoders': {'datetime_attribute': {'past': ['dayofyear']}}},
                                      model_likelihood={"likelihood": QuantileRegression([0.05, 0.1, 0.5, 0.9, 0.95])},
                                      forecast_horizon=30,
                                      site_id="ARIK",
                                      historical_covariates=True,)
forecaster_hist_test.make_forecasts()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


{'input_chunk_length': 540, 'hidden_dim': 16, 'model': 'GRU', 'n_rnn_layers': 4, 'add_encoders': {'datetime_attribute': {'past': ['dayofyear']}}}


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
`Trainer.fit` stopped: `max_epochs=500` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Predicting: 0it [00:00, ?it/s]

You are transforming a stochastic TimeSeries (i.e., contains several samples). The resulting DataFrame is a 2D object with all samples on the columns. If this is not the expected behavior consider calling a function adapted to stochastic TimeSeries like quantile_df().
